In [2]:
from warnings import filterwarnings
filterwarnings('ignore')

import pandas as pd
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import gc
import re
import operator 

import numpy as np
import pandas as pd

from gensim.models import KeyedVectors

from sklearn import model_selection

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Input, Dense, CuDNNGRU,concatenate, Bidirectional, SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import RMSprop, Adam
from keras.models import Model
from keras.callbacks import EarlyStopping

import seaborn as sns

from tqdm import tqdm
tqdm.pandas()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.width', 50000)
tqdm.pandas()
%matplotlib inline

In [ ]:
Practise is British. Practice is used in the US.

In [3]:
train = pd.read_csv('train.csv', usecols=['id','comment_text', 'target'])
test = pd.read_csv('test.csv')
print("Train shape : ",train.shape)
print("Test shape : ",test.shape)


train.head(2)

Train shape :  (1804874, 3)
Test shape :  (97320, 2)


,id,target,comment_text
0,59848,0.0,"This is so cool. It's like, 'would you want yo..."
1,59849,0.0,Thank you!! This would make my life a lot less...


In [4]:
from gensim.models import KeyedVectors
embeddings_index = KeyedVectors.load_word2vec_format('embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin', binary=True)

In [7]:
def clean_trump(word):
    if word =='tRump':
        return 'trump'
    else: 
        return word

def build_vocab(sentences):
    vocab = {}
    for sentence in tqdm(sentences,disable = (not True)):
        for word in sentence:
            if word in ['and','of','to', 'a']:
                pass
            else:
                word = clean_website_link(word)
                word = clean_trump(word)
                try:
                    vocab[word] += 1
                except KeyError:
                    vocab[word] = 1
    return vocab

import operator 

def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.3%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.3%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words


def clean_text(x):
    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '=':
        x = x.replace(punct, f' {punct} ')
    for punct in '@':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<>[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x 


def clean_website_link(site):
    if 'www' in site[:4]:
        return site[:-3][3:]
    else:
        return site
    
import re

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {'colour':'color',
                'centre':'center',
                'centres':'centers',
                'cheque':'check',
                'cheques':'checks',
                'didnt':'did not',
                'offence':'offense',
                'offences':'offenses',
                'favour':'favor',
                'favours':'favors',
                'realise':'realize',
                'doesnt':'does not',
                'defence':'defense',
                'isnt':'is not',
                'humour':'humor',
                'theatre':'theater',
                'grey':'gray',
                'neighbour':'neighbor',
                'neighbours':'neighbors',
                'neighbourhood':'neighborhood',
                'neighbourhoods':'neighborhoods',
                'shouldnt':'should not',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'honour':'honor',
                'selfies':'pictures',
                'selfie':'picture',
                'cancelled':'canceled',
                'labelled':'labeled',
                'labour':'labor',
                'licence':'license',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium',
                'signalling':'signaling',
                'travelling':'traveling',
                'judgements':'judgments',
                'cancelling':'canceling',
                'favoured':'favored',
                'colours':'colors' ,
                'organisation':'organization',
                'litre':'liter',
                'acknowledgement':'acknowledgment',
                'judgement':'judgment',
                'honourable':'honorable',
                'Koncerned':'concerned',
                'dubia':'Dubia',
                'analyses':'analysis',
                'travelled':'traveled',
                '…and':'and',
                'twittercom':'twitter',
                'Selfie':'picture',
                'favourable':'favorable',
                'recognise':'recognize',
                'behaviours':'behaviors',
                'behaviour':'behavior',
                'OBAMAcare':'obamacare',
                'coloured':'colored',
                'counselling':'counseling',
                'honoured':'honored',
                'coloured':'colored',
                'googl':'google',
                'Humanae':'Humane',
                'worshippers':'worshipers',
                'kilometres':'kilometers',
                'kilometre':'kilometer',
                'metre':'meter',
                'metres':'meters',
                'worshipper':'worshiper',
                'telegraphc':'telegraphic',
                'endeavour':'endeavor',
                'endeavours':'endeavors',
                'labelling':'labeling',
                'arse':'ass',
                'programme':'programe',
                'judgemental':'judgmental',
                '…you':'you',
                'CONservative':'conservative',
                'travellers':'travelers',
                '…the': 'the',
                'canadas':'canada',
                'TheDonald':'Donald',
                'tRUMP':'trump',
                'practise':'practice',
                'criticise':'criticize',
                'criticises':'criticizes',
                'flavour':'flavor',
                'flavours':'flavors',
                'Deplorables':'Deplorable',
                'worshipping': 'worshiping',
                'apologise':'apologize',
                'learnt':'learned',
                'harbour':'harbor',
                '…that':'that',
                'fuelled':'fueled',
                'rumours':'rumors',
                'rumour':'rumor',
                'programmes':'programes',
                'realDonaldTrump':'Trump',
                'saviour':'savior',
                'saviours':'saviors',
                'trads':'trad',
                'organised':'organized',
                'dependant':'dependent',
                'armoured':'armored',
                'fibre':'fiber',
                '…it':'it',
                'enquiry':'inquiry',
                'enquiries':'inquiries',
                'wilfully':'willfully',
                'wilfull':'willfull',
                'councillors':'councilors',
                'subsidised':'subsidized'

                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)


def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])    
    for p in punct:
        text = text.replace(p, f' {p} ')     
    return text

def known_contractions(embed):
    known = []
    for contract in contraction_mapping:
        if contract in embed:
            known.append(contract)
    return known

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])    
    for p in punct:
        text = text.replace(p, f' {p} ')     
    return text

def unknown_punct(embed, punct):
    unknown = ''
    for p in punct:
        if p not in embed:
            unknown += p
            unknown += ' '
    return unknown

punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {}
for i in unknown_punct(embeddings_index, punct):
    punct_mapping[i] = ' '

In [8]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would",
                       "he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", 
                       "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                       "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will",
                       "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", 
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                       "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                       "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have",
                       "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have",
                       "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will",
                       "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                       "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have",
                       "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                       "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                       "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [9]:
print("- Known Contractions -")
print("   Google News :")
print(known_contractions(embeddings_index))

- Known Contractions -
   Google News :
["ain't", "aren't", "can't", "could've", "couldn't", "didn't", "doesn't", "don't", "hadn't", "hasn't", "haven't", "he'd", "he'll", "he's", "how'd", "how's", "I'd", "I'd've", "I'll", "I'm", "I've", "isn't", "it'd", "it'll", "it's", "let's", "ma'am", "must've", "o'clock", "oughtn't", "she'd", "she'll", "she's", "should've", "shouldn't", "that's", "there's", "here's", "they'd", "they'll", "they're", "they've", "wasn't", "we'd", "we'll", "we're", "we've", "weren't", "what're", "what's", "what've", "where'd", "where's", "who'll", "who's", "who've", "won't", "would've", "wouldn't", "wouldn't've", "y'all", "you'd", "you'll", "you're", "you've"]


In [10]:
tr = pd.DataFrame()
tr['comment_text'] = train['comment_text']
print('Done copying to tr')
tr["comment_text"] = tr["comment_text"].apply(lambda x: clean_text(x))
print('Done cleaning text')
tr["comment_text"] = tr["comment_text"].apply(lambda x: clean_numbers(x))
print('Done cleaning numbers')
tr["comment_text"] = tr["comment_text"].apply(lambda x: replace_typical_misspell(x))
print('Done replacing mispells from between american and english language')
tr['comment_text'] = tr['comment_text'].apply(lambda x: clean_contractions(x, contraction_mapping))
print('Done cleaning contractions')
tr['comment_text'] = tr['comment_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))
print('Done cleaning special characters')
# sentences = tr["comment_text"].apply(lambda x: x.split()).values
# print('Done getting sentences')
# vocab = build_vocab(sentences)
# print('Done building the vocabulary')

Done copying to tr
Done cleaning text
Done cleaning numbers
Done replacing mispells from between american and english language
Done cleaning contractions
Done cleaning special characters


In [313]:
oov = check_coverage(vocab,embeddings_index)

Found embeddings for 37.284% of vocab
Found embeddings for  99.246% of all text


In [11]:
data = pd.concat([tr,train[['target']]],axis=1)
data.head()

,comment_text,target
0,This is so cool It s like would you want your...,0.000000
1,Thank you This would make my life a lot less a...,0.000000
2,This is such an urgent design problem kudos to...,0.000000
3,Is this something I ll be able to install on m...,0.000000
4,haha you guys are a bunch of losers,0.893617


In [12]:
del tr
del train
# del oov
# del sentences

In [13]:
data['target'] = np.where(data['target'] >= 0.5, True, False)

In [14]:
data.head(2)

,comment_text,target
0,This is so cool It s like would you want your...,False
1,Thank you This would make my life a lot less a...,False


In [15]:
train_df, validate_df = model_selection.train_test_split(data, test_size=0.15)
print('%d train comments, %d validate comments' % (len(train_df), len(validate_df)))

1534142 train comments, 270732 validate comments


In [16]:
MAX_NUM_WORDS = 100000
TOXICITY_COLUMN = 'target'
TEXT_COLUMN = 'comment_text'

In [17]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_df[TEXT_COLUMN])

In [18]:
MAX_SEQUENCE_LENGTH = 256
def pad_text(texts, tokenizer):
    return pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=MAX_SEQUENCE_LENGTH)

In [19]:
EMBEDDINGS_DIMENSION = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1,EMBEDDINGS_DIMENSION))

In [20]:
gc.collect()

66

In [21]:
num_words_in_embedding = 0

for word, i in tokenizer.word_index.items():
    if word in embeddings_index.vocab:
        embedding_vector = embeddings_index[word]
        embedding_matrix[i] = embedding_vector        
        num_words_in_embedding += 1

In [22]:
train_text = pad_text(train_df[TEXT_COLUMN], tokenizer)
train_labels = train_df[TOXICITY_COLUMN]
validate_text = pad_text(validate_df[TEXT_COLUMN], tokenizer)
validate_labels = validate_df[TOXICITY_COLUMN]

In [23]:
gc.collect()

22

In [24]:
seq_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embed_layer = Embedding(len(tokenizer.word_index) + 1,
                            EMBEDDINGS_DIMENSION,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
y = embed_layer(seq_input)
y = SpatialDropout1D(0.2)(y)
y = Bidirectional(CuDNNGRU(64, return_sequences=True))(y)   
y = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(y)

avg_pool = GlobalAveragePooling1D()(y)
max_pool = GlobalMaxPooling1D()(y)     

y = concatenate([avg_pool, max_pool])

pred = Dense(1, activation='sigmoid')(y)


m = Model(seq_input, pred)
m.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 256, 300)     122910600   input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 256, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidi

In [26]:
m.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['acc'])

In [27]:
BATCH_SIZE = 1024
NUM_EPOCHS = 100

In [ ]:
m.fit(
    train_text,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    validation_data=(validate_text, validate_labels),
    callbacks = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1534142 samples, validate on 270732 samples
Epoch 1/100
 378880/1534142 [======>.......................] - ETA: 3:03:36 - loss: 0.1914 - acc: 0.9349